In [1]:
import requests
from bs4 import BeautifulSoup
import json
from graphviz import Digraph
import pymorphy3
import time

In [2]:
morph = pymorphy3.MorphAnalyzer()
WIKIPEDIA_API_URL = "https://ru.wikipedia.org/w/api.php"
TARGET_LANGUAGE = "ru"
START_PAGE = "Информатика"
MAX_PAGES = 30

In [3]:
def get_page_data(title):
    params = {
        "action": "query",
        "format": "json",
        "titles": title,
        "prop": "extracts|links|images",
        "explaintext": True,
        "plnamespace": 0,
        "pllimit": "max",
        "imlimit": "max"
    }
    response = requests.get(WIKIPEDIA_API_URL, params=params)
    data = response.json()
    pages = data.get("query", {}).get("pages", {})
    page_data = {}
    for page_id, page in pages.items():
        page_title = page.get("title", "")
        extract = page.get("extract", "")
        links = page.get("links", [])
        images = page.get("images", [])
        page_data = {
            "title": page_title,
            "extract": extract,
            "links": [link['title'] for link in links],
            "images": [image['title'] for image in images]
        }
    return page_data

In [4]:
def normalize_title(title):
    parsed = morph.parse(title)[0]
    return parsed.normal_form.title()
def collect_data(start_title, max_pages):
    collected = {}
    to_visit = [start_title]
    visited = set()
    while to_visit and len(collected) < max_pages:
        current_title = to_visit.pop(0)
        normalized_title = normalize_title(current_title)
        if normalized_title in visited:
            continue
        print(f'Собираем данные статьи: {normalized_title}')
        page_data = get_page_data(normalized_title)
        if not page_data:
            print(f'Не удалось получить данные для статьи: {normalized_title}')
            continue
        collected[normalized_title] = page_data
        visited.add(normalized_title)
        for link in page_data['links']:
            normalized_link = normalize_title(link)
            if normalized_link not in visited and normalized_link not in to_visit:
                to_visit.append(normalized_link)
        time.sleep(0.1)
    return collected

In [5]:
def save_to_json(data, filename="wikipedia_data.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f'Данные сохранены в {filename}')

In [6]:
def create_dot_graph(data, filename="graph.dot"):
    dot = Digraph(comment='Graph of Wikipedia Articles', format='dot')
    for title in data:
        dot.node(title, title)
    for title, content in data.items():
        for link in content['links']:
            normalized_link = normalize_title(link)
            if normalized_link in data:
                dot.edge(title, normalized_link)
                dot.save(filename)
    print(f'Граф сохранен в {filename}')

In [7]:
if __name__ == "__main__":
    data = collect_data(START_PAGE, MAX_PAGES)
    save_to_json(data)
    create_dot_graph(data)

Собираем данные статьи: Информатика
Собираем данные статьи: 1948 Год
Собираем данные статьи: 1960-Е Год
Собираем данные статьи: 4 Декабрь
Собираем данные статьи: Case
Собираем данные статьи: Cgi (Графика)
Собираем данные статьи: Cmmi
Собираем данные статьи: Csab
Собираем данные статьи: Capability Maturity Model
Собираем данные статьи: Cleanroom Software Engineering
Собираем данные статьи: Communications Of The Acm
Собираем данные статьи: Computer Science And Engineering
Собираем данные статьи: Createspace
Собираем данные статьи: Dsdm
Собираем данные статьи: Doi
Собираем данные статьи: Dual Vee Model
Собираем данные статьи: Folding@Home
Собираем данные статьи: Function Model
Собираем данные статьи: Garland Science
Собираем данные статьи: Gemeinsame Normdatei
Собираем данные статьи: Http
Собираем данные статьи: Ibm
Собираем данные статьи: Ibm 704
Собираем данные статьи: Idef
Собираем данные статьи: Levy, Steven
Собираем данные статьи: Microsoft Solutions Framework
Собираем данные статьи: